# Helpful Functions

## <span style="color: cyan; font-weight:bold">Preprocessing </span>
- <span style="color: pink"><b>transforms.RandomAffine </b>(degrees, translate=None, scale=None, shear=None, interpolation='nearest', fill=0, fillcolor=None, resample=None) </span>

    <u>Documentation</u>:
    - https://pytorch.org/vision/0.9/transforms.html 

    <u>Notes:</u>
    - Random affine transformation of the image keeping center invariant
    -  If the image is torch Tensor, it is expected to have […, H, W] shape, where … means an arbitrary number of leading dimensions
    - degrees is a range of degrees to select from (set as 0 for no rotation)
    -  translate is used to translate horizontally and vertically using (a,b)
    - scale is the scaling factor (a,b) and the factor is selected within this range
    - shear is a range of degrees to choose from or a number
    - interpolation is decided using transforms.InterpolationMode (only NEAREST and BILINEAR work with tensors)
    - fill is the pixel fill value for the area outside the transformed image


- <span style="color: pink"><b>Dataloader </b>(dataset, batch_size, shuffle) </span>

    <u>Notes:</u>
    - Helps with iterating over a dataset
    - Includes batching (essential for large datasets)
    - Shuffles the data when set to True so the model doesn't learn the exact order of the dataset

## <span style="color: cyan; font-weight:bold">Training </span>
- <span style="color: pink"><b>F.one_hot </b>(y) </span>

    <u>Notes:</u>
    - Used to one hot encode 
    - Important to use with classification along with softmax/sigmoid 
    - Each label is represented as a distinct vector where a value of 1 is the label

- <span style="color: pink"><b>torch.nn.Linear </b>(in_features, out_features, bias = True) </span>

    <u>Notes:</u>
    - Creates a Dense layer with linear transformation (fully connected)
    - in_features is the size of the input sample
    - out_features is the size of the output sample
    - bias indicates whether bias should be included for the neurons in this layers

- <span style="color: pink"><b>torch.nn.MaxPool2d </b>(kernel_size, stride) </span>

    <u>Notes:</u>
    - Performs 2D max pooling over the input 
    - Downsamples the input to reduce its dimensions while maintaining the important points

- <span style="color: pink"><b>nn.ReLU </b>() </span>

    <u>Notes:</u>
    - Applies a rectified linear unit ReLU activation function

- <span style="color: pink"><b>torch.nn.Conv2d </b>(in_channels, out_channels, kernel_size, stride, padding) </span>

    <u>Notes:</u>
    - Creates a convolutional layer
    - in and out channels describe the input and output channels dimensions (3 for RGB input)


## <span style="color: cyan; font-weight:bold">Performance Metrics </span>
- <span style="color: pink"><b>accuracy_score </b>(y_real, y_predicted) </span>
- <span style="color: pink"><b>precision_score </b>(y_real, y_predicted, average='macro') </span>
- <span style="color: pink"><b>recall_score </b>(y_real, y_predicted, average='macro') </span>
- <span style="color: pink"><b>f1_score </b>(y_real, y_predicted, average='macro') </span>

# Needed Libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
import torch.nn.functional as F

from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader, random_split

import os
import tempfile
from PIL import Image

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from ray import tune, train
from ray.train import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler

from functools import partial

# Loading Datasets

Datasets can be imported from `torchvision` or used through a custom dataset class

1. Using `torchvision`
    - Import the `datasets` module from `torchvision`
    - Define a directory (`data_directory`) where the datasets will be stored
    - Import the training dataset (ex:`datasets.MNIST()`) with the following arguments:
        - `root`: The directory where the data will be stored
        - `train=True`: Indicates that we want to import the training dataset
        - `download=True`: Tells PyTorch to download the dataset if it's not already downloaded
        - `transform=None`: By default, the transform is set to `None`, which means no transformations will be applied to the data. You can specify transformations here if needed (e.g., resizing, normalization).
    - We import the testing dataset in a similar way, but with `train=False` to indicate that we want to import the testing dataset.

2. Creating your own custom dataset class
    - Based on the structure of your dataset, you need to create a class with specific functions to load the images 
    - You can then use `random_split` or any similar function to split the dataset into training and testing
    - Find below an example used for HW 3 with directories' names as the labels for the images:
        - Dimsum
        - Cookies
        - Sushi

In [4]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                images.append((img_path, self.class_to_idx[cls]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# CNN 

The result size of the convolution is determined by the formula:

$$\text{{Result size}} = \frac{{W - F + 2P}}{S} + 1$$

where

- $W$: Input size
- $F$: Filter size
- $P$: Padding
- $S$: Stride

# Ray Tune

Steps:
- Modify the class to include the chosen parameters to be tuned
- Make a function for training 
    - The function takes the search space or config as a parameter
    - The training part is exactly the same as normal training
    - Before training, define the model using the config parameters (shown below)
    - Do not start the training automatically from epoch 0. Check the checkpoint and determine if the starting epoch is 0 or something else (shown below)
- Define a function for testing 
    - Not neccesary but makes it easier 
    - Takes the model as a parameter
- Define the search space as needed
- Define the tuner with the required parameters (shown below)
- Fit the tuner to get the results
- Obtain the best results and create a model using these results
- Train the best model and analyze its performance

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Modify the class to include the chosen parameters to be tuned
class Model(nn.Module):
    def __init__(self, l1=10):
        super(Model.self).__init__()

        self.network = nn.Sequential(
            nn.Linear(3*32*32, l1, bias=True),
            nn.ReLU(),
            nn.Linear(l1, 3)
        )
    def forward(self, x):
        x = torch.flatten(x,1)
        x = self.network(x)
        return x
    
# Make a function for training
def train_model(config):
    model = Model(config["l1"])
    model = model.to(device)
    criterion = nn.CrossEntropy()
    optimizer = optim.SGD(model.parameters(), lr=config['lr'])
    checkpoint = train.get_checkpoint()
    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        model.load_state_dict(checkpoint_state["model_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    for epoch in range(start_epoch,10):
        running_loss = 0.0
        for (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
       
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(val_dataloader, 0):
                    with torch.no_grad():
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)

                        outputs = model(inputs)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                        loss = criterion(outputs, labels)
                        val_loss += loss.cpu().numpy()
                        val_steps += 1

        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            checkpoint = None
            if (i + 1) % 5 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(temp_checkpoint_dir, "model.pth")
                )
                checkpoint = Checkpoint.from_directory(temp_checkpoint_dir)

            train.report({"accuracy": (100 * correct / total)}, checkpoint=checkpoint)

# Make a function for testing
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    precision = precision_score(labels, predicted, average='macro')
    recall = recall_score(labels, predicted, average='macro')
    f1 = f1_score(labels, predicted, average='macro')

    return (100 * correct / total), precision, recall, f1

# Define the search space
config = {
    "l1": tune.choice([2 ** i for i in range(6)]),
    "lr": tune.loguniform(1e-4, 1e-1)
}

# Define the tuner
tuner = tune.Tuner(
  train_model,
  tune_config=tune.TuneConfig(
      num_samples=10,
      scheduler=ASHAScheduler(metric="accuracy", mode="max"),
  ),
  param_space=config,
)

# Fit the tuner
result = tuner.fit()

# Use the best results to get the best model
best_result = result.get_best_result("accuracy", mode="max")
with best_result.checkpoint.as_directory() as checkpoint_dir:
    state_dict = torch.load(os.path.join(checkpoint_dir, "model.pth"))

model_tuned = Model(best_result.config['l1'])
model_tuned = model_tuned.to(device)

# Continue by training the model and checking its performance